In [30]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [31]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
SEED = 42
tf.get_logger().setLevel('ERROR')
print('Tensorflow: ' + tf.__version__)
print('GPU is', 'available' if tf.config.experimental.list_physical_devices('GPU') else 'not available')

Tensorflow: 2.4.0
GPU is available


In [32]:
df_train = pd.read_csv('nlp-getting-started/train.csv')
df_test = pd.read_csv('nlp-getting-started/test.csv')
df_submission = pd.read_csv('nlp-getting-started/sample_submission.csv')

seed = 42

X_train, X_valid, y_train, y_valid = train_test_split(
    df_train['text'].tolist(), 
    df_train['target'].tolist(), 
    test_size=0.2, 
    stratify = df_train['target'].tolist(), 
    random_state=seed)


train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train))
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid,y_valid))


In [23]:
# num_threads = 4
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.shuffle(len(train_ds))
# train_ds = train_ds.map(parse_function, num_parallel_calls=num_threads)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

valid_ds = valid_ds.batch(batch_size)
valid_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)

In [33]:
train_ds.element_spec

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [8]:
X_train = df_train.text.values
y_train = df_train.target.values
X_test = df_test.text.values

In [10]:
X_train

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [39]:
tfhub_preprocessor = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2'
# tfhub_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1'
tfhub_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"

# tfhub_preprocessor = 'https://tfhub.dev/tensorflow/albert_en_preprocess/2'
# tfhub_encoder = 'https://tfhub.dev/tensorflow/albert_en_base/2'

In [40]:
def embeddings(inputs):
    
    preprocessor = hub.KerasLayer(tfhub_preprocessor, name='preprocessing')
    encoder_inputs = preprocessor(inputs)
    encoder = hub.KerasLayer(tfhub_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    
    return outputs

In [19]:
# Quick test output of preprocessing

# text_test = ['this is such an amazing movie!']
text_test = [X_train[0]]
preprocessor = hub.KerasLayer(tfhub_preprocessor, name='preprocessing')
text_preprocessed = preprocessor(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_type_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [  101  2256 15616  2024  1996  3114  1997  2023  1001  8372  2089 16455]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [20]:
# Quick test output of the BERT encoder
encoder = hub.KerasLayer(tfhub_encoder)
bert_results = encoder(text_preprocessed)

print(f'Loaded BERT: {tfhub_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[-0.3659554   0.8747347  -0.984028   -0.92114395 -0.9974049   0.4714694
  0.02822361  0.31982425  0.07394147 -0.37385422  0.17668313 -0.20407893]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[ 0.04336619  0.07277332  0.6108962  ...  0.9239689  -1.5104988
   0.05670904]
 [-0.69057775 -0.01006321  1.4062763  ...  0.0357721  -1.6613307
  -0.4464908 ]
 [ 0.36303955  0.15109812  0.66403073 ...  0.88282275 -1.6734009
  -0.2836811 ]
 ...
 [ 0.74721557 -0.14383739  1.4926044  ...  1.3075274  -0.8495825
  -0.48952693]
 [-0.08880976 -0.21782646  1.022673   ...  0.33986157 -0.97045386
  -0.40322617]
 [ 0.08066449  1.1958051   1.1949328  ...  1.6354626  -1.3808879
  -0.43884376]]


In [11]:
# text_test = ['this is such an amazing movie!']
bert_results = embeddings(X_train[:2])

print(f'Loaded BERT: {tfhub_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * {'input_type_ids': <tf.Tensor 'inputs_1:0' shape=(2, 128) dtype=int32>, 'input_word_ids': <tf.Tensor 'inputs_2:0' shape=(2, 128) dtype=int32>, 'input_mask': <tf.Tensor 'inputs:0' shape=(2, 128) dtype=int32>}
    * False
    * None
  Keyword arguments: {}

Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * [TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids')]
    * False
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * [TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/

In [41]:
def build_model():
    
    inputs = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessor = hub.KerasLayer(tfhub_preprocessor, name='preprocessing')
    
    encoder_inputs = preprocessor(inputs)
    encoder = hub.KerasLayer(tfhub_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
    
    model = tf.keras.Model(inputs=inputs, outputs=net)
    
    return model

In [42]:
model = build_model()
model.summary()

ValueError: in user code:

    C:\Users\Henri\anaconda3\envs\ml\lib\site-packages\tensorflow_hub\keras_layer.py:235 call  *
        result = smart_cond.smart_cond(training,
    C:\Users\Henri\anaconda3\envs\ml\lib\site-packages\tensorflow\python\saved_model\load.py:668 _call_attribute  **
        return instance.__call__(*args, **kwargs)
    C:\Users\Henri\anaconda3\envs\ml\lib\site-packages\tensorflow\python\eager\def_function.py:828 __call__
        result = self._call(*args, **kwds)
    C:\Users\Henri\anaconda3\envs\ml\lib\site-packages\tensorflow\python\eager\def_function.py:871 _call
        self._initialize(args, kwds, add_initializers_to=initializers)
    C:\Users\Henri\anaconda3\envs\ml\lib\site-packages\tensorflow\python\eager\def_function.py:725 _initialize
        self._stateful_fn._get_concrete_function_internal_garbage_collected(  # pylint: disable=protected-access
    C:\Users\Henri\anaconda3\envs\ml\lib\site-packages\tensorflow\python\eager\function.py:2969 _get_concrete

In [37]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=[tf.keras.metrics.BinaryAccuracy()])

In [38]:
%%time

history = model.fit(train_ds,
                    epochs=3,
                    validation_data=valid_ds,
                    verbose=2)

Epoch 1/3


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  Trying to access resource using the wrong type. Expected class tensorflow::lookup::LookupInterface got class tensorflow::lookup::LookupInterface
	 [[{{node model_2/preprocessing/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/bert_tokenizer/StatefulPartitionedCall/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets}}]]
	 [[model_2/preprocessing/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/bert_pack_inputs/PartitionedCall/pad_model_inputs_1/ones_like_1/Shape/_150]]
  (1) Invalid argument:  Trying to access resource using the wrong type. Expected class tensorflow::lookup::LookupInterface got class tensorflow::lookup::LookupInterface
	 [[{{node model_2/preprocessing/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/bert_tokenizer/StatefulPartitionedCall/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets/Wor

In [14]:
X_train[:2]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada'], dtype=object)

In [14]:
bert_results["pooled_output"][0, :12]

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([-0.36595532,  0.8747347 , -0.984028  , -0.92114407, -0.99740493,
        0.4714695 ,  0.02822366,  0.31982425,  0.07394186, -0.37385425,
        0.1766832 , -0.20407897], dtype=float32)>